In [ ]:
import math
import numpy as np
from numpy.random import shuffle, seed, choice
from tqdm import tqdm
from collections import defaultdict, Counter
import pandas as pd
import re
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
from torch.utils.data import random_split,Dataset,DataLoader
from torchvision import datasets, transforms
from torch import nn, optim

import torchvision.transforms as T

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [ ]:
import os

directory = 'sample_data/'

concatenated_content = ''

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            concatenated_content += file.read() + '\n'

print(f"Total number of characters in concatenated files: {len(concatenated_content)}")

Total number of characters in concatenated files: 116615


In [ ]:
def preprocess_code(code):
    code = re.sub(r'(?<=[^\n])  +', ' ', code)
    code = '\n'.join(line.strip() for line in code.splitlines())
    return code


In [ ]:
concatenated_content = preprocess_code(concatenated_content)

In [ ]:
print(concatenated_content[:1000])


import java.util.Random;

public class HashTableExperiment {

private static int count = 0;



/******************
* Helper Methods *
******************/
private static boolean less(int v, int w) {
++count;
return (v < w);
}

private static boolean equal(int v, int w) {
++count;
return (v == w);
}

private class Node {
int item;
Node next;
public Node(int k, Node n) {
item = k;
next = n;
}

}

/***************************
* Our methods for the lab *
***************************/

private static Random rnd = new Random();

public static int[] genRandomArray(int size){

int a[] = new int[size];
for(int i = 0; i < a.length; i++){
a[i] = rnd.nextInt();
}
return a;
}

public static int[] genSortedArray(int size){
int a[] = new int[size];
for(int i = 0; i < a.length; i++){
a[i] = i;
}
return a;
}

// create a best case tree for the integers 1 .. N, by doing binary search exhaustively
// on all ints and entering nodes into the tree

private static Node genBestBST(int N) {
return genBestBSTHel

In [ ]:
size = max(20000, len(concatenated_content))
text = concatenated_content[:size]

chars_in_text = sorted(list(set(text)))
char2int = {ch: i for i, ch in enumerate(chars_in_text)}
int2char = {i: ch for i, ch in enumerate(chars_in_text)}
num_chars = len(chars_in_text)

sample_len = 300
input_seq_chars = []
target_seq_chars = []

for i in range(0, len(text) - sample_len, 1):
    input_seq_chars.append(text[i:i + sample_len - 1])
    target_seq_chars.append(text[i + 1:i + sample_len])

NameError: name 'concatenated_content' is not defined

In [ ]:
input_seq = [[char2int[ch] for ch in seq] for seq in input_seq_chars]
target_seq = [[char2int[ch] for ch in seq] for seq in target_seq_chars]

def seq2OneHot(seq, num_chars):
    return np.array([np.eye(num_chars)[s] for s in seq], dtype=np.float32)

input_seq = seq2OneHot(input_seq, num_chars)
target_seq = seq2OneHot(target_seq, num_chars)

input_seq = torch.tensor(input_seq, dtype=torch.float32)
target_seq = torch.tensor(target_seq, dtype=torch.float32)

In [ ]:
input_seq.shape, target_seq.shape

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
class Basic_Dataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

ds = Basic_Dataset(input_seq, target_seq)
batch_size = 128
data_loader = DataLoader(ds, batch_size=batch_size, shuffle=True)

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, dropout):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(device)
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc1(out)
        return out

hidden_dim = 128
n_layers = 1
dropout = 0.1
model = Model(num_chars, num_chars, hidden_dim, n_layers, dropout).to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)

In [ ]:
num_epochs = 50
losses = []
from tqdm import tqdm

model.train()
for epoch in tqdm(range(num_epochs)):
    for input_seq_batch, target_seq_batch in data_loader:
        input_seq_batch, target_seq_batch = input_seq_batch.to(device), target_seq_batch.to(device)
        optimizer.zero_grad()
        target_seq_hat = model(input_seq_batch)
        loss = loss_fn(target_seq_hat, target_seq_batch.view(-1, num_chars))
        loss.backward()
        optimizer.step()
    losses.append(loss.item())

# Plotting the loss
plt.title('Loss over epochs')
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()


In [ ]:
def int2OneHot(X,size):

    def int2OneHot1(x,size=10):
        tmp = np.zeros(size)
        tmp[int(x)] = 1.0
        return tmp

    return np.array([ int2OneHot1(x, size) for x in X ]).astype('float32')
def softmax_with_temperature(logits, temperature):
    scaled_logits = logits / temperature
    softmax = F.softmax(scaled_logits, dim=0)
    return softmax.detach().cpu().numpy()

def predict(model, ch, temperature=0.3):
    ch = ch[-(sample_len - 1):]
    ch = np.array([char2int[c] for c in ch])
    ch = np.array([int2OneHot(ch, num_chars)])
    ch = torch.from_numpy(ch).to(device)
    out = model(ch)
    prob = softmax_with_temperature(out[-1], temperature)
    char_ind = np.random.choice(list(range(num_chars)), p=prob)
    return int2char[char_ind]

def sample(model, out_len, start, temperature=0.3):
    model.eval()
    chars = [ch for ch in start]
    size = out_len - len(chars)
    for _ in range(size):
        char = predict(model, ''.join(chars), temperature)
        chars.append(char)
    return ''.join(chars)

In [ ]:
generated_code = sample(model, 300, "import java.awt.Color;\nimport java.utils.Random;\n public ", temperature=0.3)
print(generated_code)